In [1]:
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve,auc,classification_report,confusion_matrix
from sklearn.model_selection import KFold,train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale

In [2]:
df=pd.read_csv("10%data.csv")
y=df["status_time"]
newdf = pd.DataFrame(scale(df), index=df.index, columns=df.columns)
newdf=newdf.round(3)
df=newdf
df.drop("id",axis=1,inplace=True)

In [7]:
x = df.drop(columns = ["status_time","investor_orig_time","REtype_SF_orig_time","REtype_PU_orig_time","REtype_CO_orig_time"])

In [8]:
x

,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,uer_time,...,gdp_orig,gdp_dif,gdp_inc,uer_orig,uer_avg,uer_dif,uer_inc,hpi_avg,hpi_dif,hpi_inc
0,-0.628,0.408,-0.073,0.200,-0.418,-0.097,-0.662,1.237,0.704,-0.729,...,0.321,0.586,0.432,0.049,-0.650,-0.689,-0.684,1.115,0.195,-0.002
1,-0.825,-0.655,-0.845,-0.163,0.612,-0.471,-0.173,1.085,0.748,-0.564,...,0.750,0.433,0.348,0.928,-0.191,-0.792,-0.763,0.247,1.408,1.181
2,-0.530,0.009,-0.227,0.139,-0.234,-0.508,-0.786,1.196,0.333,-0.729,...,0.648,0.048,0.119,0.049,-0.630,-0.689,-0.684,1.061,0.659,0.364
3,-1.022,-1.585,-0.381,-0.586,-1.098,-1.920,1.116,0.631,0.927,-0.453,...,0.724,0.631,0.462,0.401,-0.242,-0.534,-0.533,0.452,1.889,2.120
4,-1.219,-0.788,-0.999,-0.163,0.245,-0.178,-0.354,0.023,0.789,-0.288,...,1.597,0.084,0.207,1.104,0.098,-0.586,-0.578,-0.553,0.869,0.692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4167,0.650,0.806,0.699,-0.646,-0.417,0.684,3.104,-1.113,-2.393,1.921,...,-1.647,-1.732,-3.237,-0.654,0.676,1.990,2.159,-0.669,-1.329,-1.047
4168,1.437,0.806,0.699,0.562,-0.964,1.112,0.288,-1.223,-0.151,1.645,...,-1.647,0.604,0.418,-0.654,2.125,1.733,1.885,-1.158,-1.398,-1.094
4169,0.552,0.806,0.854,0.502,-0.686,1.960,0.992,-1.302,-2.793,1.645,...,-1.128,-2.390,-2.971,-0.830,0.460,1.784,-0.533,-0.726,-1.446,-1.127
4170,1.929,-0.389,-0.690,-0.042,-1.039,0.353,-0.601,-1.119,-0.105,0.982,...,0.413,-0.300,-0.122,0.928,1.523,0.651,0.524,-0.767,-0.349,-0.454


In [9]:
df

,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,uer_time,...,gdp_dif,gdp_inc,uer_orig,uer_avg,uer_dif,uer_inc,hpi_avg,hpi_dif,hpi_inc,status_time
0,-0.628,0.408,-0.073,0.200,-0.418,-0.097,-0.662,1.237,0.704,-0.729,...,0.586,0.432,0.049,-0.650,-0.689,-0.684,1.115,0.195,-0.002,0.755
1,-0.825,-0.655,-0.845,-0.163,0.612,-0.471,-0.173,1.085,0.748,-0.564,...,0.433,0.348,0.928,-0.191,-0.792,-0.763,0.247,1.408,1.181,0.755
2,-0.530,0.009,-0.227,0.139,-0.234,-0.508,-0.786,1.196,0.333,-0.729,...,0.048,0.119,0.049,-0.630,-0.689,-0.684,1.061,0.659,0.364,0.755
3,-1.022,-1.585,-0.381,-0.586,-1.098,-1.920,1.116,0.631,0.927,-0.453,...,0.631,0.462,0.401,-0.242,-0.534,-0.533,0.452,1.889,2.120,0.755
4,-1.219,-0.788,-0.999,-0.163,0.245,-0.178,-0.354,0.023,0.789,-0.288,...,0.084,0.207,1.104,0.098,-0.586,-0.578,-0.553,0.869,0.692,0.755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4167,0.650,0.806,0.699,-0.646,-0.417,0.684,3.104,-1.113,-2.393,1.921,...,-1.732,-3.237,-0.654,0.676,1.990,2.159,-0.669,-1.329,-1.047,-1.324
4168,1.437,0.806,0.699,0.562,-0.964,1.112,0.288,-1.223,-0.151,1.645,...,0.604,0.418,-0.654,2.125,1.733,1.885,-1.158,-1.398,-1.094,-1.324
4169,0.552,0.806,0.854,0.502,-0.686,1.960,0.992,-1.302,-2.793,1.645,...,-2.390,-2.971,-0.830,0.460,1.784,-0.533,-0.726,-1.446,-1.127,-1.324
4170,1.929,-0.389,-0.690,-0.042,-1.039,0.353,-0.601,-1.119,-0.105,0.982,...,-0.300,-0.122,0.928,1.523,0.651,0.524,-0.767,-0.349,-0.454,-1.324


In [10]:
y

0       1
1       1
2       1
3       1
4       1
       ..
4167    0
4168    0
4169    0
4170    0
4171    0
Name: status_time, Length: 4172, dtype: int64

In [11]:
kf=KFold(n_splits=5,random_state=2,shuffle=True)

In [13]:
train_x, test_x, train_y, test_y=train_test_split(x, y, train_size=0.7)

In [20]:
model = svm.SVC(kernel='poly',gamma=1) 
model.fit(train_x, train_y)

SVC(gamma=1, kernel='poly')

In [21]:
model.score(test_x, test_y)

0.7044728434504792

In [22]:
pred_y = model.predict(test_x)
print(confusion_matrix(test_y, pred_y))

[[269 163]
 [207 613]]


In [23]:
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.57      0.62      0.59       432
           1       0.79      0.75      0.77       820

    accuracy                           0.70      1252
   macro avg       0.68      0.69      0.68      1252
weighted avg       0.71      0.70      0.71      1252

